In [1]:
from PIL import Image, ImageFont, ImageDraw
import numpy as np
import cv2
import io
import math

In [24]:
target_image = 'images/monalisa.jpeg'

In [30]:
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from scipy import spatial
import random
import uuid

def resize_image(img : Image, size : tuple) -> np.ndarray:
        resz_img = ImageOps.fit(img, size, Image.LANCZOS, centering=(0.5, 0.5))
        return np.array(resz_img)

def blend_image(region, tile, opacity_percent):
    alpha = opacity_percent/100 # 1.0 - (color_diff / 255.0)
    blended_region = (alpha * region + (1.0 - alpha) * tile).astype(np.uint8)
    return blended_region

def find_closest_images(given_image, image_list, k=3):
    image_array = np.stack(image_list)
    mse_values = np.mean((image_array - given_image) ** 2, axis=(1, 2, 3))
    closest_indices = np.argpartition(mse_values, k)[:k]
    closest_images = [image_list[idx] for idx in closest_indices]
    return random.choice(closest_images)

def remove_black_borders(image_array):
    mask = np.any(image_array != [0, 0, 0], axis=-1)
    coords = np.argwhere(mask)
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1
    cropped_image_array = image_array[y0:y1, x0:x1]
    return cropped_image_array

def image_mosaic_v1(target_image_path, tile_images_path, output_dir, divisions=200, scale=1, opacity_percent=50):
    target_image = Image.open(target_image_path)
    target_image = target_image.convert("RGB")
    original_width, original_height = target_image.size
    print('target image size', target_image.size)
    print('image mode', target_image.mode)
    target_image_resized = resize_image(target_image, (original_width * scale, original_height * scale))
    target_image_array = np.array(target_image_resized)
    grid_size = (target_image_array.shape[0] // divisions, target_image_array.shape[1] // divisions)

    
    # Preprocess tile images
    tile_images = [np.array(resize_image(Image.open(tile_path).convert("RGB"), (grid_size[1], grid_size[0]))) for tile_path in tile_images_path]

    combined_image = np.zeros_like(target_image_array)

    for i in range(divisions):
        for j in range(divisions):
            x1, y1 = j * grid_size[1], i * grid_size[0]
            x2, y2 = (j + 1) * grid_size[1], (i + 1) * grid_size[0]

            grid_image = target_image_array[y1:y2, x1:x2, :]
            # print(f'grid image', grid_image.shape)
            # print(f'tile images', tile_images[0].shape)
            # print(f'grid_size ',grid_size)
            closest_image = find_closest_images(grid_image, tile_images)
            blended_image = blend_image(grid_image, closest_image,opacity_percent)
            combined_image[y1:y2, x1:x2, :] = blended_image

    cropped_image = remove_black_borders(combined_image)
    img = Image.fromarray(cropped_image)

    output_path = f'{str(uuid.uuid4())}_mosaic_opacity{opacity_percent}.png'
    img.save(output_path)
    return output_path

In [31]:
import glob 
tile_path = glob.glob('shape_images/*.png')

In [37]:
image_mosaic_v1('images/sampleImage.jpeg', tile_path[:50], '', opacity_percent=60, divisions=100)

target image size (800, 600)
image mode RGB


'1972eea7-4405-4ade-9c71-25e3e1f6d4b7_mosaic_opacity60.png'

In [27]:
import cv2
import numpy as np

def remove_black_borders(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply binary thresholding to create a binary image
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(contours[0])
    for contour in contours:
        x_, y_, w_, h_ = cv2.boundingRect(contour)
        x = min(x, x_)
        y = min(y, y_)
        w = max(w, x_ + w_ - x)
        h = max(h, y_ + h_ - y)

    # Crop the image using the bounding box coordinates
    cropped_image = image[y:y+h, x:x+w]

    return cropped_image

# Usage
image_path = 'c5b4efe0-32f4-473b-bdb9-a99570e1caba_mosaic_opacity60.png'
cropped_image = remove_black_borders(image_path)


# Save the cropped image if needed
cv2.imwrite('cropped_image.jpg', cropped_image)


True

In [29]:
from PIL import Image
import numpy as np

def remove_black_borders(image_path, output_path):
    # Open the image
    image = Image.open(image_path)
    
    # Convert the image to grayscale
    gray_image = image.convert('L')
    
    # Convert grayscale image to numpy array
    gray_array = np.array(gray_image)
    
    # Create a binary mask where black pixels are 1 and others are 0
    mask = gray_array > 0
    
    # Find the bounding box coordinates of the non-black area
    coords = np.argwhere(mask)
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1  # slices are exclusive at the top
    
    # Crop the image using the bounding box coordinates
    cropped_image = image.crop((x0, y0, x1, y1))
    
    # Save the cropped image
    cropped_image.save(output_path)
    
    return cropped_image

# Usage
image_path = 'c5b4efe0-32f4-473b-bdb9-a99570e1caba_mosaic_opacity60.png'
output_path = 'cropped_imag1e.jpg'
cropped_image = remove_black_borders(image_path, output_path)


In [35]:
from PIL import Image, ImageDraw
import io
import os
import random
import math

def generate_shape_image(shape, color):
    # Create a new image with a transparent background
    width, height = 10, 10
    image = Image.new('RGBA', (width, height), (255, 255, 255, 255))

    # Draw the specified shape with the specified color
    draw = ImageDraw.Draw(image)
    if shape == 'rectangle':
        draw.rectangle((2, 2, width-2, height-2), fill=color)
    elif shape == 'triangle':
        draw.polygon([(2, height-2), (width-2, height-2), (width//2, 2)], fill=color)
    elif shape == 'ellipse':
        draw.ellipse((2, 2, width-2, height-2), fill=color)
    elif shape == 'circle':
        radius = min(width, height) // 2 - 2
        draw.ellipse((width//2 - radius, height//2 - radius, width//2 + radius, height//2 + radius), fill=color)
    elif shape == 'pentagon':
        draw.polygon(polygon_vertices(5, width//2, height//2, width//2 - 2), fill=color)
    elif shape == 'hexagon':
        draw.polygon(polygon_vertices(6, width//2, height//2, width//2 - 2), fill=color)

    # Save the image to a buffer
    img_buffer = io.BytesIO()
    image.save(img_buffer, format='PNG')

    # Get the image data as bytes
    img_bytes = img_buffer.getvalue()

    return img_bytes

def polygon_vertices(sides, center_x, center_y, radius):
    """Generate the vertices of a regular polygon."""
    angle = 2 * math.pi / sides
    return [(center_x + radius * math.cos(i * angle), center_y + radius * math.sin(i * angle)) for i in range(sides)]

# Create a folder to save the images if it doesn't exist
if not os.path.exists('shape_images'):
    os.makedirs('shape_images')

# Define shapes and colors
shapes = ['rectangle', 'triangle', 'ellipse', 'circle', 'pentagon', 'hexagon']
colors = [
    (255, 0, 0, 255), (0, 255, 0, 255), (0, 0, 255, 255),
    (255, 255, 0, 255), (0, 255, 255, 255), (255, 0, 255, 255),
    (128, 0, 0, 255), (0, 128, 0, 255), (0, 0, 128, 255),
    (128, 128, 0, 255), (0, 128, 128, 255), (128, 0, 128, 255),
    (255, 165, 0, 255), (255, 20, 147, 255), (0, 191, 255, 255),
    (144, 238, 144, 255), (221, 160, 221, 255), (255, 182, 193, 255),
    (127, 255, 212, 255), (240, 230, 140, 255), (189, 183, 107, 255),
    (255, 105, 180, 255), (72, 61, 139, 255), (199, 21, 133, 255),
    (65, 105, 225, 255), (238, 130, 238, 255), (255, 228, 181, 255),
    (152, 251, 152, 255), (135, 206, 250, 255), (255, 99, 71, 255),
    (219, 112, 147, 255), (60, 179, 113, 255), (255, 248, 220, 255)
]

# Generate and save 20 shape images with random shapes and colors
for i in range(20):
    # Choose a random shape and color
    shape = random.choice(shapes)
    color = random.choice(colors)

    # Generate the shape image with the specified shape and color
    shape_image = generate_shape_image(shape, color)

    # Save the image to a file in the 'shape_images' folder
    file_path = f'shape_images/image_{i}.png'
    with open(file_path, 'wb') as file:
        file.write(shape_image)

    print(f"Image {i} ({shape}) saved.")

print("All images saved in 'shape_images' folder.")


Image 0 (triangle) saved.
Image 1 (hexagon) saved.
Image 2 (triangle) saved.
Image 3 (rectangle) saved.
Image 4 (triangle) saved.
Image 5 (rectangle) saved.
Image 6 (pentagon) saved.
Image 7 (rectangle) saved.
Image 8 (circle) saved.
Image 9 (circle) saved.
Image 10 (triangle) saved.
Image 11 (triangle) saved.
Image 12 (triangle) saved.
Image 13 (rectangle) saved.
Image 14 (hexagon) saved.
Image 15 (ellipse) saved.
Image 16 (hexagon) saved.
Image 17 (circle) saved.
Image 18 (hexagon) saved.
Image 19 (triangle) saved.
All images saved in 'shape_images' folder.
